In [1]:
%pylab inline
import numpy as np
import pandas as pd
data = []
datapath = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/CN_Futures_2014.01.29.csv'
data = pd.read_csv(datapath)

Populating the interactive namespace from numpy and matplotlib


In [2]:
data.head()

,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity
0,CNF14,1,2014-01-29D04:24:59.489814800,9672009CE4FAC9FA,1,689500,2,,A,689500,2
1,CNF14,2,2014-01-29D04:24:59.489814800,2D8DE96295907A90,1,662000,50,,B,662000,50
2,CNF14,3,2014-01-29D08:45:01.488515400,037E4D2362CE450E,1,662500,1,,B,662500,1
3,CNF14,4,2014-01-29D08:45:01.592505000,037E4D2362CE450E,1,662500,-1,,B,662000,50
4,CNF14,5,2014-01-29D08:45:01.689504000,03FE4D231081F2C2,1,667500,1,,B,667500,1


In [3]:
data_CNF14 = data[data.Series == 'CNF14']
data = data_CNF14

In [4]:
import time
start = time.time()

order_book_bid = []
order_book_ask = []
timestamp = data.TimeStamp[0]
temp_ask = 0
temp_bid = 0
bid = []
ask = []
timestamp_ = []
x = data.TimeStamp.unique()
#a = range(0,len(data.TimeStamp.unique()),1000)
time_contain = []
time_book_sorting = []
timestamp_.append(x[0])
y = data[data['TimeStamp'].str.contains(x[0])]
bid.append(y[(y.BidOrAsk == 'B')][["Price","OrderNumber","QuantityDifference"]]) # bid
ask.append(y[(y.BidOrAsk == 'A')][["Price","OrderNumber","QuantityDifference"]]) # ask
a = bid[0].sort(['Price'],ascending = [False])
b = ask[0].sort(['Price'],ascending = [True])    
order_book_bid.append(a[a.QuantityDifference != 0])
order_book_ask.append(b[b.QuantityDifference != 0])

for i in range(len(y),3200,1):    
    if data.BidOrAsk[i] == 'A' :
        #print i 
        #print temp_ask
        #print x[temp_ask]
        #print data[i:i+1][['Price',"OrderNumber","QuantityDifference"]]
        #print data.TimeStamp[i] == x[temp_ask]
        #print data[i:i+1][['QuantityDifference']] > 0,int(data[['QuantityDifference']][i:i+1].values)  
        if int(data[['QuantityDifference']][i:i+1].values) > 0 :
            if data.TimeStamp[i] == x[temp_ask]:            
                #print len(order_book_ask[temp_ask][(order_book_ask[temp_ask].Price == int(data[i:i+1][['Price']].values))]) >= 1                 
                if len(order_book_ask[temp_ask][(order_book_ask[temp_ask].Price == int(data[i:i+1][['Price']].values))]) >= 1:
                
                    legth_same_price = len(order_book_ask[temp_ask][(order_book_ask[temp_ask].Price == int(data[i:i+1][['Price']].values))])
                    #print order_book_ask[temp_ask][(order_book_ask[temp_ask].Price == int(data[i:i+1][['Price']].values))].index[0]
                    #print legth_same_price
                    #print order_book_ask[temp_ask]
                    position_ = order_book_ask[temp_ask][(order_book_ask[temp_ask].Price == int(data[i:i+1][['Price']].values))].index[0] + legth_same_price + 1
                    #print position_
                    Order_Position = data[i:i+1][['OrderBookPosition']].iloc[0].values
                    #print Order_Position
                    ##print position_ == Order_Position
                    order_book_ask[temp_ask] = pd.concat([order_book_ask[temp_ask],data[i:i+1][['Price',"OrderNumber",'QuantityDifference']]])
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].sort(['Price'],ascending = [True])
                else:
                    position_ = order_book_ask[temp_ask][(order_book_ask[temp_ask].Price == int(data[i:i+1][['Price']].values))].index + 1
                    #print position_
                    order_book_ask[temp_ask] = pd.concat([order_book_ask[temp_ask],data[i:i+1][['Price',"OrderNumber",'QuantityDifference']]])
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].sort(['Price'],ascending = [True])
                    order_book_ask[temp_ask] = pd.DataFrame(order_book_ask[temp_ask].values,columns = ['Price',"OrderNumber",'QuantityDifference'])                
                    #print order_book_ask[temp_ask]
                    position_ = order_book_ask[temp_ask][(order_book_ask[temp_ask].Price == int(data[i:i+1][['Price']].values))].index + 1
                    Order_Position = data[i:i+1][['OrderBookPosition']].iloc[0].values
                    ##print position_ == Order_Position
                #print '\t' 
            
            elif data.TimeStamp[i] != x[temp_ask]:
                #print len(order_book_ask[temp_ask][(order_book_ask[temp_ask].Price == int(data[i:i+1][['Price']].values))]) >= 1
                
                if len(order_book_ask[temp_ask][(order_book_ask[temp_ask].Price == int(data[i:i+1][['Price']].values))]) >= 1:
                    legth_same_price = len(order_book_ask[temp_ask][(order_book_ask[temp_ask].Price == int(data[i:i+1][['Price']].values))])                
                    order_book_ask.append(0)
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = pd.concat([order_book_ask[temp_ask - 1],data[i:i+1][['Price',"OrderNumber",'QuantityDifference']]])
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].sort(['Price'],ascending = [True])
                    order_book_ask[temp_ask] = pd.DataFrame(order_book_ask[temp_ask].values,columns = ['Price',"OrderNumber",'QuantityDifference'])                
                    position_ = order_book_ask[temp_ask][(order_book_ask[temp_ask].Price == int(data[i:i+1][['Price']].values))].index[0] + legth_same_price + 1
                    #print position_
                    Order_Position = data[i:i+1][['OrderBookPosition']].iloc[0].values
                    #print legth_same_price
                    #print order_book_ask[temp_ask]
                    #print int(data[i:i+1][['Price']].values)
                    ##print position_ == Order_Position
                else:
                    order_book_ask.append(0)
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = pd.concat([order_book_ask[temp_ask - 1],data[i:i+1][['Price',"OrderNumber",'QuantityDifference']]])
                    #print order_book_ask[temp_ask]
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].sort(['Price'],ascending = [True])
                    order_book_ask[temp_ask] = pd.DataFrame(order_book_ask[temp_ask].values,columns = ['Price',"OrderNumber",'QuantityDifference'])
                    #print order_book_ask[temp_ask]
                    #print int(data[i:i+1][['Price']].values)
                    position_ = order_book_ask[temp_ask][(order_book_ask[temp_ask].Price == int(data[i:i+1][['Price']].values))].index + 1
                    ##print position_ == data[i:i+1][['OrderBookPosition']].iloc[0]                                       

                order_book_ask[temp_ask] = order_book_ask[temp_ask].sort(['Price'],ascending = [True])
                #print '\t'                                          
                 
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:
            Price_ =  int(data[['Price']][i:i+1].values)
            Order_Number_ = data[['OrderNumber']][i:i+1].iloc[0] 
            position_ = int(data[['OrderBookPosition']][i:i+1].values)
            index_ = order_book_ask[temp_ask][(order_book_ask[temp_ask].Price == Price_)].index + 1
            #if len(index_[index_ == position_]) == 1:
            #    print [True]
                
            #print index_
            #print position_
            #print Order_Number_ 
            #print order_book_ask[temp_ask]
            
            drop_values_ = pd.concat([order_book_ask[temp_ask],data[i:i+1][['Price',"OrderNumber",'QuantityDifference']]])
            
            order_book_ask[temp_ask] = drop_values_[['Price',"OrderNumber",'QuantityDifference']].groupby(by = ['Price','OrderNumber'],as_index = False)['QuantityDifference'].sum()
            #print order_book_ask[temp_ask]
            order_book_ask[temp_ask] = order_book_ask[temp_ask][order_book_ask[temp_ask].QuantityDifference != 0]
            order_book_ask[temp_ask] = pd.DataFrame(order_book_ask[temp_ask].values,columns = ['Price',"OrderNumber",'QuantityDifference'])
            #print order_book_ask[temp_ask]
            #print '\t' 

    elif data.BidOrAsk[i] == 'B' :
        #print i 
        #print temp_ask
        #print x[temp_ask]
        #print data[i:i+1][['Price',"OrderNumber","QuantityDifference"]]
        #print data.TimeStamp[i] == x[temp_ask]
        #print data[i:i+1][['QuantityDifference']] > 0,int(data[['QuantityDifference']][i:i+1].values)
        
        if int(data[['QuantityDifference']][i:i+1].values) > 0 :
            if data.TimeStamp[i] == x[temp_bid]:            
                #print len(order_book_bid[temp_bid][(order_book_bid[temp_bid].Price == int(data[i:i+1][['Price']].values))]) >= 1                 
                if len(order_book_bid[temp_bid][(order_book_bid[temp_bid].Price == int(data[i:i+1][['Price']].values))]) >= 1:
                
                    legth_same_price = len(order_book_bid[temp_bid][(order_book_ask[temp_bid].Price == int(data[i:i+1][['Price']].values))])
                    #print order_book_bid[temp_bid][(order_book_ask[temp_bid].Price == int(data[i:i+1][['Price']].values))].index[0]
                    #print legth_same_price
                    #print order_book_bid[temp_bid]
                    position_ = order_book_bid[temp_bid][(order_book_bid[temp_bid].Price == int(data[i:i+1][['Price']].values))].index[0] + legth_same_price + 1
                    #print position_
                    Order_Position = data[i:i+1][['OrderBookPosition']].iloc[0].values
                    #print Order_Position
                    ##print position_ == Order_Position
                    order_book_bid[temp_bid] = pd.concat([order_book_bid[temp_bid],data[i:i+1][['Price',"OrderNumber",'QuantityDifference']]])
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].sort(['Price'],ascending = [False])
                else:
                    position_ = order_book_bid[temp_bid][(order_book_bid[temp_bid].Price == int(data[i:i+1][['Price']].values))].index + 1
                    #print position_
                    order_book_bid[temp_bid] = pd.concat([order_book_bid[temp_bid],data[i:i+1][['Price',"OrderNumber",'QuantityDifference']]])
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].sort(['Price'],ascending = [False])
                    order_book_bid[temp_bid] = pd.DataFrame(order_book_bid[temp_bid].values,columns = ['Price',"OrderNumber",'QuantityDifference'])                
                    #print order_book_bid[temp_bid]
                    position_ = order_book_bid[temp_bid][(order_book_bid[temp_bid].Price == int(data[i:i+1][['Price']].values))].index + 1
                    Order_Position = data[i:i+1][['OrderBookPosition']].iloc[0].values
                    ##print position_ == Order_Position
                #print '\t' 
            
            elif data.TimeStamp[i] != x[temp_bid]:
                #print len(order_book_bid[temp_bid][(order_book_bid[temp_bid].Price == int(data[i:i+1][['Price']].values))]) >= 1
                
                if len(order_book_bid[temp_bid][(order_book_bid[temp_bid].Price == int(data[i:i+1][['Price']].values))]) >= 1:
                    legth_same_price = len(order_book_bid[temp_bid][(order_book_bid[temp_bid].Price == int(data[i:i+1][['Price']].values))])                
                    order_book_bid.append(0)
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = pd.concat([order_book_bid[temp_bid - 1],data[i:i+1][['Price',"OrderNumber",'QuantityDifference']]])
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].sort(['Price'],ascending = [False])
                    order_book_bid[temp_bid] = pd.DataFrame(order_book_bid[temp_bid].values,columns = ['Price',"OrderNumber",'QuantityDifference'])                
                    position_ = order_book_bid[temp_bid][(order_book_bid[temp_bid].Price == int(data[i:i+1][['Price']].values))].index[0] + legth_same_price + 1
                    #print position_
                    Order_Position = data[i:i+1][['OrderBookPosition']].iloc[0].values
                    #print legth_same_price
                    #print order_book_ask[temp_ask]
                    #print int(data[i:i+1][['Price']].values)
                    ##print position_ == Order_Position
                else:
                    order_book_bid.append(0)
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = pd.concat([order_book_bid[temp_bid - 1],data[i:i+1][['Price',"OrderNumber",'QuantityDifference']]])
                    #print order_book_bid[temp_bid]
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].sort(['Price'],ascending = [False])
                    order_book_bid[temp_bid] = pd.DataFrame(order_book_bid[temp_bid].values,columns = ['Price',"OrderNumber",'QuantityDifference'])
                    #print order_book_bid[temp_bid]
                    #print int(data[i:i+1][['Price']].values)
                    position_ = order_book_bid[temp_bid][(order_book_bid[temp_bid].Price == int(data[i:i+1][['Price']].values))].index + 1
                    ##print position_ == data[i:i+1][['OrderBookPosition']].iloc[0]                                       

                order_book_bid[temp_bid] = order_book_bid[temp_bid].sort(['Price'],ascending = [False])
                #print '\t'                                          
                 
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:
            Price_ =  int(data[['Price']][i:i+1].values)
            Order_Number_ = data[['OrderNumber']][i:i+1].iloc[0] 
            position_ = int(data[['OrderBookPosition']][i:i+1].values)
            index_ = order_book_bid[temp_bid][(order_book_bid[temp_bid].Price == Price_)].index + 1
            #if len(index_[index_ == position_]) == 1:
            #    print [True]
                
            #print index_
            #print position_
            #print Order_Number_ 
            #print order_book_ask[temp_ask]
            
            drop_values_ = pd.concat([order_book_bid[temp_bid],data[i:i+1][['Price',"OrderNumber",'QuantityDifference']]])
            
            order_book_bid[temp_bid] = drop_values_[['Price',"OrderNumber",'QuantityDifference']].groupby(by = ['Price','OrderNumber'],as_index = False)['QuantityDifference'].sum()
            #print order_book_bid[temp_bid]
            order_book_bid[temp_bid] = order_book_bid[temp_bid][order_book_bid[temp_bid].QuantityDifference != 0]
            order_book_bid[temp_bid] = pd.DataFrame(order_book_bid[temp_bid].values,columns = ['Price',"OrderNumber",'QuantityDifference'])
            #print order_book_bid[temp_bid]
            #print '\t'             
end = time.time()            
print "Total time = %f"%(end - start)

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:20: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:21: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:173: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:180: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:159: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:83: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipyker

Total time = 57.582674


In [5]:
import time
start = time.time()
order_book_bid_merge = []
order_book_ask_merge = []
for i in range(0,len(order_book_bid)):
    a = order_book_bid[i][['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False).sum().sort(['Price'],ascending = [False])
    order_book_bid_merge.append(pd.DataFrame(a.values))
for i in range(0,len(order_book_ask)):
    a = order_book_ask[i][['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False).sum()#.sort(['Price'],ascending = [True])
    order_book_ask_merge.append(pd.DataFrame(a.values))
end = time.time()            
print "Total time = %f"%(end - start) 

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Total time = 18.388858


In [6]:
 order_book_bid_merge[100]

,0,1
0,674000,3
1,673500,28
2,673000,48
3,672500,41
4,672000,29
5,671500,48
6,671000,58
7,670500,22
8,670000,1
9,669500,1


In [7]:
 order_book_ask_merge[100]

,0,1
0,675500,9
1,676000,46
2,676500,52
3,677000,77
4,677500,57
5,678000,41
6,678500,26
7,679000,23
8,679500,3
9,680000,25
